In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
import pickle

def get_mood_api():
    
    reviewmood = pickle.load(open('review.pkl','rb'))
    
    def mood_data_api(reviewmood,data):
        df ={}
        df[0] = data
        return pd.DataFrame.from_dict(df, orient='index')
        
        df_suprv = df.drop(df.columns[[2,3,4,5,6,9,10,11]], axis =1)
        #df_suprv.loc[:,'reviewType']=df_suprv['overall'].apply(lambda x: 1 if x >= 4.0 else 0 )
        df_suprv.loc[:,'verified'] = [1 if x=='True' else 0  for x in df_suprv['verified']]
        def count_punct(text):
            count = sum([1 for char in text if char in string.punctuation])
            ratio = count/(len(text) - text.count(" "))
            return round(ratio, 4)*100

        df_suprv['punct%'] = df_suprv['reviewText'].apply(lambda x: count_punct(x))
        df_suprv['body_len'] = df_suprv['reviewText'].apply(lambda x: len(x) - x.count(" "))
        df_suprv.rename(columns ={'summary' :'Summary', 'overall' :'Overall'},inplace = True)
        tfidf_vect = TfidfVectorizer(ngram_range=(2,2))
        tfidf_vect1 = TfidfVectorizer(ngram_range=(2,2))
        X_tfidf = tfidf_vect.fit_transform(df_suprv['cleaned_text'])
        XS_tfidf = tfidf_vect1.fit_transform(df_suprv['cleaned_summ'])
        X_tfidf_df = pd.DataFrame(X_tfidf.toarray())
        X_tfidf_df.columns = tfidf_vect.get_feature_names()
        X_tfidf_df1 = pd.DataFrame(XS_tfidf.toarray())
        X_tfidf_df1.columns = tfidf_vect1.get_feature_names()
        df_new = pd.concat([df_suprv, X_tfidf_df,X_tfidf_df1],axis=1,sort=False)
        df_new.fillna(value =0, inplace = True)
        
        ReviewType = reviewmood.predict(df_new)
        return ReviewType
    
    return mood_data_api


In [13]:
import json
from flask import Flask

app = Flask(__name__)
mood_api = get_mood_api()

#Post method
@app.route('/reviews', methods =['POST'])
def reviews():
    input_data = request.json

    output_data = mood_api(input_data)
    response = json.dumps(output_data)
    return response



ModuleNotFoundError: No module named 'Ensemble1'